In [1]:
def replace_col_name(df, new, old):
    
    if new == '':
        del df[old]
        return
    
    df[new] = df[old]
    del df[old]

In [20]:
import excelHelper
import importlib, pandas, sys, re, os
importlib.reload(excelHelper)
helper = excelHelper.excelHelper()

def get_df(fname, sheetname=None, index=False):
    
    if sheetname is None:
        df = pandas.read_excel(fname, index=index)
    else:
        df = pandas.read_excel(fname, sheetname=sheetname, index=index)
        
    print(df.head())
    
    if 'baseMean' in df.columns:
        df['Mean frequency'] = df['baseMean']
        del df['baseMean']
    
    for col in ['lfcSE', 'pvalue', 'stat']:
        del df[col]

    df['Gene name'] = df.index
    if 'padj' in df.columns:
        df['p value (adjusted)'] = df['padj']
        del df['padj']
    
    df = df[['Gene name', 'Mean frequency', 'log2FoldChange', 'p value (adjusted)']]
    return df
in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S4 raw DESeq2 output.xlsx'
#df = get_df(in_file, sheetname=None, index=True)

import volcanoMaker
print('---')
importlib.reload(volcanoMaker)
v = volcanoMaker.volcanoMaker()
v.gl_rnaseq()
v.read_counts_files(
    dirname='/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/counts_6_reps/')
print('---')
v.read_clip_deseq_excel(in_file)
v.read_sp_vs_oo_as_programs()
print(v.clipdf.head())
print('---')
v.filter_read_counts(lower_cutoff_average=20)
v.table_of_stats()



volcanoMaker.write_excel_of_deseq(v.clipdf.copy(), header='SPvOO')
helper.add_gene_description(v.clipdf)
print('\n>>>\n', v.clipdf)

v.clipdf = v.clipdf[v.clipdf['has_ortiz']].copy()
    
replace_col_name(v.clipdf, 'Gene name', 'gene_name')    
replace_col_name(v.clipdf, 'p value (adjusted)', 'padj')
replace_col_name(v.clipdf, '', 'gene_id')
replace_col_name(v.clipdf, 'Mean frequency', 'baseMean')


v.clipdf = v.clipdf[['Gene name', 'Mean frequency', 'log2FoldChange', 'p value (adjusted)', 'Description']]


oo_up = v.clipdf[v.clipdf['log2FoldChange']<=-1].copy()
oo_up = oo_up[oo_up['p value (adjusted)']<=0.01].copy()

sp_up = v.clipdf[v.clipdf['log2FoldChange']>=1].copy()
sp_up = sp_up[sp_up['p value (adjusted)']<=0.01].copy()

print('oo_up index, ', len(oo_up.index), 'sp_up index, ', len(sp_up.index))



---
---
From Nobel et al., loaded 6655 SP genes, 9415 OO genes, and 12839 total GL genes.
CLIP peaks in a GL program: Oogenic only                 4798
Oogenic and Spermatogenic    2462
Spermatogenic only           1825
                             1036
Name: Program, dtype: int64
            baseMean  log2FoldChange     lfcSE       stat        pvalue  \
clec-87   477.187118       -4.007144  0.211601 -18.937294  5.621309e-80   
snpc-1.3  415.796892        4.524025  0.262932  17.206052  2.391951e-66   
mrp-4     338.825486       -5.008864  0.302134 -16.578275  1.000607e-61   
meg-1     313.353018       -5.908888  0.409290 -14.436936  3.030150e-47   
fog-1     923.307044        3.030068  0.211325  14.338428  1.258741e-46   

                  padj gene_name         gene_id     Wormbase ID  has_ortiz  \
clec-87   5.689327e-76   clec-87  WBGene00007709  WBGene00007709       True   
snpc-1.3  1.210447e-62  snpc-1.3  WBGene00017265  WBGene00017265       True   
mrp-4     3.375715e-58     mrp

In [22]:
import os
os.getcwd()
print('\n&*&*&*\n', v.clipdf.head())


&*&*&*
          Gene name  Mean frequency  log2FoldChange  p value (adjusted)  \
clec-87    clec-87      477.187118       -4.007144        5.689327e-76   
snpc-1.3  snpc-1.3      415.796892        4.524025        1.210447e-62   
mrp-4        mrp-4      338.825486       -5.008864        3.375715e-58   
meg-1        meg-1      313.353018       -5.908888        7.667038e-44   
fog-1        fog-1      923.307044        3.030068        2.547943e-43   

                                                Description  
clec-87         C-type lectin domain-containing protein 87   
snpc-1.3  SNAPc (Small Nuclear RNA Activating Complex) h...  
mrp-4                                                        
meg-1                                                        
fog-1              FOG-1 protein; Feminization Of Germline   


In [25]:
        
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S4 DESeq2 SP vs OO.xlsx')

def widen_column(sheet_name):
    worksheet = writer.sheets[sheet_name]
#    workbook  = writer.book

    worksheet.set_column('A:A', 20)
    worksheet.set_column('B:B', 20)
    worksheet.set_column('C:C', 20)
    worksheet.set_column('D:D', 20)

sp_up.to_excel(
    writer,
    sheet_name='SP enriched',
    index=False,
    #columns=tab3_columns
)

oo_up.to_excel(
    writer,
    sheet_name='OO enriched',
    index=False,
    #columns=tab3_columns
)

v.clipdf.to_excel(
    writer,
    sheet_name='All DESeq2 results',
    index=False,

)
print(oo_up)

for col in ['SP enriched', 'OO enriched', 'All DESeq2 results']:
    widen_column(col)
writer.save()

           Gene name  Mean frequency  log2FoldChange  p value (adjusted)  \
clec-87      clec-87      477.187118       -4.007144        5.689327e-76   
mrp-4          mrp-4      338.825486       -5.008864        3.375715e-58   
meg-1          meg-1      313.353018       -5.908888        7.667038e-44   
cpg-2          cpg-2      714.042490       -2.325217        1.150347e-34   
puf-5          puf-5     1125.227613       -2.304972        6.051913e-30   
pos-1          pos-1      310.546115       -2.368572        2.355870e-27   
F14H3.6      F14H3.6       70.918500       -5.426825        2.627319e-26   
W05F2.3      W05F2.3      225.569157       -2.252567        2.417593e-25   
die-1          die-1      105.013750       -3.510844        4.800231e-25   
cpg-3          cpg-3       95.356401       -3.457000        4.962231e-23   
C01G8.1      C01G8.1      447.998958       -2.035978        8.089950e-23   
spn-4          spn-4      450.659576       -2.264145        1.096432e-22   
clec-91     

In [14]:



in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/Table S6 Blocks.xls'

xls = pandas.ExcelFile(in_file)
print(xls.sheet_names)

df_by_sheetname = {}
for sheetname in xls.sheet_names:
    df = pandas.read_excel(in_file, sheetname=sheetname, index=False)
    helper.add_gene_description(df)
    del df['transcript_id']
    print(df)
    df_by_sheetname[sheetname] = df.copy()
    
    
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S6 Blocks.xlsx')

for sheetname, df in df_by_sheetname.items():
    df.to_excel(
        writer,
        sheet_name=sheetname,
        index=False,
        #columns=tab3_columns
    )
    
writer.save()
#print(df.head(2))

['Block I', 'Block II', 'Block III']
      Gene name                                        Description
0       enu-3.5                        ENhancer of Uncoordination 
1         rpl-2                          60S ribosomal protein L8 
2        col-94                                          COLlagen 
3       F22B8.7                                                   
4         fog-3           FOG-3 protein; Feminization Of Germline 
5       R11F4.1                          Probable glycerol kinase 
6      Y47D3A.5                                                   
7      R04B5.11                                                   
8       Y4C6A.4                                                   
9      C09H10.9                                                   
10      F32B5.4                                MS Related Protein 
11      C44B9.2                                                   
12     F52F12.5                                                   
13     F47D12.7  Kelch re

In [ ]:

in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/orthos/output/File S7 conservation.xls'

xls = pandas.ExcelFile(in_file)
print(xls.sheet_names)


df_by_sheetname = {}
for sheetname in xls.sheet_names:
    df = pandas.read_excel(in_file, sheetname=sheetname, index=False)
    df['Gene name'] = df['Worm gene name']
    
    helper.add_gene_description(df)
    
    df['Worm gene description'] = df['Description']
    
    for col in ['transcript_id', 'Description', 'Gene name']:
        del df[col]
        
    print(df)
    df_by_sheetname[sheetname] = df.copy()
    
    
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/xFile S6 Blocks.xlsx')

for sheetname, df in df_by_sheetname.items():
    df.to_excel(
        writer,
        sheet_name=sheetname,
        index=False,
        #columns=tab3_columns
    )
    
writer.save()
#print(df.head(2))